In [1]:
from typing import Union
import google.generativeai as genai
import os
from dotenv import load_dotenv, find_dotenv
import logging

c:\Users\achakravarti\AppData\Local\anaconda3\envs\general\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
def add_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Adds two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.

    Returns:
        Union[int, float]: Sum of the two numbers.
    """
    return a + b

def sub_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Subtracts two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.
    """
    return a - b

def mul_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Multiplies two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.

    Returns:
        Union[int, float]: Product of the two numbers.
    """
    return a * b

In [33]:
# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load environment variables
if not load_dotenv(find_dotenv()):
    logging.warning("Could not find .env file. Make sure it exists and contains the required API key.")

# Fetch API key
API_KEY = os.getenv("GOOGLE_API_KEY")
if not API_KEY:
    raise ValueError("Missing GOOGLE_API_KEY in environment variables.")

# Configure GenAI
genai.configure(api_key=API_KEY)

def get_completion(system_prompt: str, user_prompt: str, model: str = "gemini-2.0-flash") -> str:
    """
    Generates a response using Google's Gemini API.

    Args:
        system_prompt (str): The system prompt providing context.
        user_prompt (str): The user's input prompt.
        model (str): The AI model to use (default: gemini-2.0-flash).

    Returns:
        str: The AI-generated response.
    """
    try:
        model_instance = genai.GenerativeModel(model)
        full_prompt = f"{system_prompt}\n\n{user_prompt}"
        response = model_instance.generate_content(full_prompt)

        if not response or not hasattr(response, "text"):
            raise ValueError("Invalid response format from API.")
    
        # print(response)
        return response.text.strip()
    
    except Exception as e:
        logging.error(f"Error generating response: {e}")
        return "An error occurred while generating the response."

def main():
    system_prompt = '''You are an AI agent who is helping in solving mathematical problems. You will be given Python \
    functions to use for problem-solving. Understand those python functions. As a reponse, you should only return return only the \
    function call so that I can Python's built-in exec(). You should not return any other text and do not include "```json" in your response.

    Below are the functions that you can use to solve the problem. Function are enclosed within tripple angular brackets.
    <<<
    def add_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
        """
        Adds two numbers and returns the result.

        Args:
            a (Union[int, float]): First number.
            b (Union[int, float]): Second number.

        Returns:
            Union[int, float]: Sum of the two numbers.
        """
        return a + b

    def sub_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
        """
        Subtracts two numbers and returns the result.

        Args:
            a (Union[int, float]): First number.
            b (Union[int, float]): Second number.
        """
        return a - b

    def mul_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
        """
        Multiplies two numbers and returns the result.

        Args:
            a (Union[int, float]): First number.
            b (Union[int, float]): Second number.

        Returns:
            Union[int, float]: Product of the two numbers.
        """
        return a * b
    >>>

    You need to format output in JSON as shown below.

    user: add two numbers 10 and 20
    model: add_two_numbers(a=10, b=20)
    
    user: multiply two numbers 10 and 20
    model: mul_two_numbers(a=10, b=20)

'''

    user_prompt = "Multiply two numbers 10 and 20"

    response = get_completion(system_prompt, user_prompt)
    print(f'Model response: {response}, type of response: {type(response)}')

    # Pass the function to exec's environment
    exec_globals = {
        'add_two_numbers': add_two_numbers,
        'mul_two_numbers': mul_two_numbers,
        'sub_two_numbers': sub_two_numbers
    }
     
    exec(f"result = {response}", exec_globals)
    result = exec_globals['result']
    print(f"Result of execution: {result}")

if __name__ == "__main__":
    main()


Model response: mul_two_numbers(a=10, b=20), type of response: <class 'str'>
Result of execution: 200


In [35]:
import os
import json
import logging
import google.generativeai as genai
from dotenv import load_dotenv, find_dotenv
from typing import Union

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load environment variables
if not load_dotenv(find_dotenv()):
    logging.warning("Could not find .env file. Make sure it exists and contains the required API key.")

# Fetch API key
API_KEY = os.getenv("GOOGLE_API_KEY")
if not API_KEY:
    raise ValueError("Missing GOOGLE_API_KEY in environment variables.")

# Configure GenAI
genai.configure(api_key=API_KEY)

def add_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Adds two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.

    Returns:
        Union[int, float]: Sum of the two numbers.
    """
    return a + b

def sub_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Subtracts two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.

    Returns:
        Union[int, float]: Difference of the two numbers.
    """
    return a - b

# Function registry
function_registry = {
    "add_two_numbers": add_two_numbers,
    "sub_two_numbers": sub_two_numbers,
}

def get_completion(system_prompt: str, user_prompt: str, model: str = "gemini-2.0-flash") -> str:
    """
    Generates a response using Google's Gemini API.

    Args:
        system_prompt (str): The system prompt providing context.
        user_prompt (str): The user's input prompt.
        model (str): The AI model to use (default: gemini-2.0-flash).

    Returns:
        str: The AI-generated response.
    """
    try:
        model_instance = genai.GenerativeModel(model)
        full_prompt = f"{system_prompt}\n\n{user_prompt}"
        response = model_instance.generate_content(full_prompt)

        if not response or not hasattr(response, "text"):
            raise ValueError("Invalid response format from API.")

        return response.text.strip()
    
    except Exception as e:
        logging.error(f"Error generating response: {e}")
        return "An error occurred while generating the response."

def select_and_execute_function(user_input: str):
    """
    Determines the appropriate function based on user input, retrieves arguments, and executes the function.

    Args:
        user_input (str): The user's request.

    Returns:
        The function result or an error message.
    """
    system_prompt = """You are an AI that selects the most appropriate function based on user input.
    Available functions:
    
    1. add_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]
       Adds two numbers and returns the result.
    
    2. sub_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]
       Subtracts two numbers and returns the result.
    
    Return JSON in the following format and do not include ```json or any markdown edits: 
    {"function": "function_name", "args": [arg1, arg2]}"""

    llm_response = get_completion(system_prompt, user_input)

    try:
        response_data = json.loads(llm_response)
        func_name = response_data["function"]
        args = response_data["args"]

        if func_name in function_registry:
            return function_registry[func_name](*args)
        else:
            raise ValueError(f"Function {func_name} not found.")
    except json.JSONDecodeError:
        logging.error(f"Failed to parse LLM response: {llm_response}")
        return "Error: Could not understand the response from AI."
    except Exception as e:
        logging.error(f"Execution error: {e}")
        return "Error: Something went wrong while executing the function."

# Example usage
user_query = "What is 10 minus 4?"
result = select_and_execute_function(user_query)
print(f"Result: {result}")


Result: 6


In [39]:
import os
import json
import logging
import inspect
import google.generativeai as genai
from dotenv import load_dotenv, find_dotenv
from typing import Union

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load environment variables
if not load_dotenv(find_dotenv()):
    logging.warning("Could not find .env file. Make sure it exists and contains the required API key.")

# Fetch API key
API_KEY = os.getenv("GOOGLE_API_KEY")
if not API_KEY:
    raise ValueError("Missing GOOGLE_API_KEY in environment variables.")

# Configure GenAI
genai.configure(api_key=API_KEY)

def add_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Adds two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.

    Returns:
        Union[int, float]: Sum of the two numbers.
    """
    return a + b

def sub_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Subtracts two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.
    """
    return a - b

def mul_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Multiplies two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.

    Returns:
        Union[int, float]: Product of the two numbers.
    """
    return a * b

# Function registry
function_registry = {
    "add_two_numbers": add_two_numbers,
    "sub_two_numbers": sub_two_numbers,
}

def extract_function_metadata(func_dict):
    """
    Extract function signatures and docstrings dynamically.

    Args:
        func_dict (dict): Dictionary of function names mapped to functions.

    Returns:
        str: Formatted string containing function metadata.
    """
    metadata_str = ""
    for name, func in func_dict.items():
        signature = str(inspect.signature(func))  # Get function signature
        docstring = inspect.getdoc(func) or "No description available."  # Extract docstring
        metadata_str += f"{name}{signature} - {docstring}\n\n"

    return metadata_str

def get_completion(system_prompt: str, user_prompt: str, model: str = "gemini-2.0-flash") -> str:
    """
    Generates a response using Google's Gemini API.

    Args:
        system_prompt (str): The system prompt providing context.
        user_prompt (str): The user's input prompt.
        model (str): The AI model to use (default: gemini-2.0-flash).

    Returns:
        str: The AI-generated response.
    """
    try:
        model_instance = genai.GenerativeModel(model)
        full_prompt = f"{system_prompt}\n\n{user_prompt}"
        response = model_instance.generate_content(full_prompt)

        if not response or not hasattr(response, "text"):
            raise ValueError("Invalid response format from API.")

        return response.text.strip()
    
    except Exception as e:
        logging.error(f"Error generating response: {e}")
        return "An error occurred while generating the response."

def select_and_execute_function(user_input: str):
    """
    Determines the appropriate function based on user input, retrieves arguments, and executes the function.

    Args:
        user_input (str): The user's request.

    Returns:
        The function result or an error message.
    """
    function_metadata = extract_function_metadata(function_registry)

    print(function_metadata)

    system_prompt = f"""You are an AI that selects the most appropriate function based on user input.
    Available functions:\n\n{function_metadata}
    
    Return JSON in the following format:
    {{"function": "function_name", "args": [arg1, arg2]}}"""

    llm_response = get_completion(system_prompt, user_input)

    try:
        response_data = json.loads(llm_response)
        func_name = response_data["function"]
        args = response_data["args"]

        if func_name in function_registry:
            return function_registry[func_name](*args)
        else:
            raise ValueError(f"Function {func_name} not found.")
    except json.JSONDecodeError:
        logging.error(f"Failed to parse LLM response: {llm_response}")
        return "Error: Could not understand the response from AI."
    except Exception as e:
        logging.error(f"Execution error: {e}")
        return "Error: Something went wrong while executing the function."

# Example usage
user_query = "What is 10 minus 4?"
result = select_and_execute_function(user_query)
print(f"Result: {result}")


add_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float] - Adds two numbers and returns the result.

Args:
    a (Union[int, float]): First number.
    b (Union[int, float]): Second number.

Returns:
    Union[int, float]: Sum of the two numbers.

sub_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float] - Subtracts two numbers and returns the result.

Args:
    a (Union[int, float]): First number.
    b (Union[int, float]): Second number.


Result: 6


In [37]:
metadata_str

NameError: name 'metadata_str' is not defined

In [33]:
from typing import Any, Callable, Dict, List, Optional, Union
import inspect
import json
import logging
import re
from dataclasses import dataclass
from enum import Enum

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FunctionType(Enum):
    MATH = "math"
    UTILITY = "utility"
    SYSTEM = "system"

@dataclass
class FunctionMetadata:
    name: str
    description: str
    parameters: Dict[str, Dict[str, Any]]
    returns: Dict[str, Any]
    type: FunctionType

class FunctionRegistry:
    def __init__(self):
        self._functions: Dict[str, Callable] = {}
        self._metadata: Dict[str, FunctionMetadata] = {}
    
    def register(self, func_type: FunctionType):
        """Decorator to register functions with metadata"""
        def decorator(func: Callable):
            sig = inspect.signature(func)
            doc = inspect.getdoc(func) or ""
            
            # Extract parameter information
            parameters = {}
            for name, param in sig.parameters.items():
                param_type = param.annotation.__name__ if param.annotation != inspect.Parameter.empty else "Any"
                parameters[name] = {
                    "type": param_type,
                    "description": "",
                    "required": param.default == inspect.Parameter.empty
                }
            
            # Create function metadata
            metadata = FunctionMetadata(
                name=func.__name__,
                description=doc.split('\n')[0],
                parameters=parameters,
                returns={"type": sig.return_annotation.__name__ if sig.return_annotation != inspect.Signature.empty else "Any"},
                type=func_type
            )
            
            self._functions[func.__name__] = func
            self._metadata[func.__name__] = metadata
            return func
        return decorator
    
    def get_function(self, name: str) -> Optional[Callable]:
        return self._functions.get(name)
    
    def get_metadata(self, name: str) -> Optional[FunctionMetadata]:
        return self._metadata.get(name)
    
    def list_functions(self, func_type: Optional[FunctionType] = None) -> List[str]:
        if func_type:
            return [name for name, meta in self._metadata.items() if meta.type == func_type]
        return list(self._functions.keys())

class AIFunctionAgent:
    def __init__(self, model_client: Any, registry: FunctionRegistry):
        self.model = model_client
        self.registry = registry
    
    def _parse_response(self, response_text: str) -> Dict[str, Any]:
        """
        Parse the model's response into a structured format.
        Handles both JSON and non-JSON responses.
        """
        # First try parsing as JSON
        try:
            return json.loads(response_text)
        except json.JSONDecodeError:
            pass
        
        # If not JSON, try to extract function name and parameters
        try:
            # Look for function name pattern
            func_match = re.search(r'(add|sub|mul)_two_numbers', response_text)
            if not func_match:
                raise ValueError("No valid function found in response")
            
            func_name = func_match.group(0)
            
            # Look for numbers in the text
            numbers = re.findall(r'\d+(?:\.\d+)?', response_text)
            if len(numbers) < 2:
                raise ValueError("Could not find two numbers in response")
            
            # Convert to appropriate type (int or float)
            params = [
                float(num) if '.' in num else int(num)
                for num in numbers[:2]
            ]
            
            return {
                "function": func_name,
                "parameters": {
                    "a": params[0],
                    "b": params[1]
                }
            }
        except Exception as e:
            logger.error(f"Error parsing non-JSON response: {e}")
            raise ValueError(f"Could not parse response: {response_text}")
    
    def _create_system_prompt(self) -> str:
        """Creates a system prompt describing available functions"""
        function_descriptions = []
        for name in self.registry.list_functions():
            metadata = self.registry.get_metadata(name)
            if metadata:
                function_descriptions.append(
                    f"Function: {name}\n"
                    f"Description: {metadata.description}\n"
                    f"Parameters: {', '.join(metadata.parameters.keys())}\n"
                )
        
        return f"""You are an AI assistant that helps execute mathematical functions. 
Available functions:

{'\n'.join(function_descriptions)}

When responding, either:
1. Return a JSON object in this format:
{{
    "function": "function_name",
    "parameters": {{
        "a": first_number,
        "b": second_number
    }}
}}

2. Or simply state the function name and parameters clearly in plain text.
For example: "Use add_two_numbers with a=10 and b=20"
"""

    def execute(self, user_prompt: str) -> Any:
        """Processes user prompt and executes the appropriate function"""
        try:
            # Get model's function selection
            system_prompt = self._create_system_prompt()
            response = self.model.generate_content(f"{system_prompt}\n\nUser request: {user_prompt}")
            
            logger.info(f"Model response: {response.text}")
            
            # Parse response
            function_call = self._parse_response(response.text)
            
            # Validate function exists
            func_name = function_call["function"]
            function = self.registry.get_function(func_name)
            if not function:
                raise ValueError(f"Function '{func_name}' not found")
            
            # Execute function
            result = function(**function_call["parameters"])
            return result
            
        except Exception as e:
            logger.error(f"Error executing function: {e}")
            raise

# Example usage
registry = FunctionRegistry()

@registry.register(FunctionType.MATH)
def add_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """Adds two numbers and returns the result."""
    return a + b

@registry.register(FunctionType.MATH)
def sub_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """Subtracts two numbers and returns the result."""
    return a - b

@registry.register(FunctionType.MATH)
def mul_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """Multiplies two numbers and returns the result."""
    return a * b

def main():
    # Initialize the model client
    model = genai.GenerativeModel("gemini-2.0-flash")
    
    # Create the agent
    agent = AIFunctionAgent(model, registry)
    
    # Execute a function based on user prompt
    try:
        result = agent.execute("substract two numbers 10 and 20")
        print(f"Result: {result}")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

2025-02-10 16:59:47,509 - INFO - Model response: Use sub_two_numbers with a=10 and b=20



Result: -10


this is fantastic code. Works really great. 
However, for me it is very difficult to understand. And I would like to extend this functionality. I may add many more functions with verity of input and outputs. 

can you make it simpler?

In [34]:
import logging
from typing import Any, Dict, Callable
import json

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class SimpleFunctionAgent:
    def __init__(self, model_client):
        self.model = model_client
        self.functions: Dict[str, Callable] = {}
        self.descriptions: Dict[str, str] = {}
    
    def add_function(self, func: Callable, description: str = None):
        """
        Register a new function with the agent.
        
        Args:
            func: The function to register
            description: A description of what the function does
        """
        func_name = func.__name__
        self.functions[func_name] = func
        self.descriptions[func_name] = description or func.__doc__ or "No description available"
        logger.info(f"Registered function: {func_name}")
    
    def create_prompt(self) -> str:
        """Create the system prompt with available functions"""
        function_list = []
        for name, desc in self.descriptions.items():
            function_list.append(f"Function: {name}\nDescription: {desc}\n")
        
        return f"""You are an AI assistant that helps execute functions.
Available functions:

{'\n'.join(function_list)}

Please respond with the function name and its parameters in this simple format:
FUNCTION: function_name
PARAMS: param1=value1, param2=value2
"""

    def parse_response(self, response_text: str) -> tuple[str, dict]:
        """
        Parse the model's response to get function name and parameters.
        
        Args:
            response_text: The raw text response from the model
            
        Returns:
            tuple: (function_name, parameters_dict)
        """
        try:
            # Split response into lines and clean them
            lines = [line.strip() for line in response_text.split('\n') if line.strip()]
            
            # Extract function name and parameters
            func_name = None
            params = {}
            
            for line in lines:
                if line.startswith('FUNCTION:'):
                    func_name = line.replace('FUNCTION:', '').strip()
                elif line.startswith('PARAMS:'):
                    # Parse parameters string into dictionary
                    params_str = line.replace('PARAMS:', '').strip()
                    if params_str:
                        # Split by comma and create dictionary
                        param_pairs = [p.strip() for p in params_str.split(',')]
                        for pair in param_pairs:
                            key, value = pair.split('=')
                            # Try to convert value to int or float if possible
                            try:
                                value = int(value)
                            except ValueError:
                                try:
                                    value = float(value)
                                except ValueError:
                                    pass
                            params[key.strip()] = value
            
            if not func_name:
                raise ValueError("No function name found in response")
            
            return func_name, params
            
        except Exception as e:
            logger.error(f"Error parsing response: {e}")
            raise ValueError(f"Could not parse response: {response_text}")

    def execute(self, user_prompt: str) -> Any:
        """
        Execute a function based on the user's prompt.
        
        Args:
            user_prompt: The user's request
            
        Returns:
            The result of the function execution
        """
        try:
            # Get model's response
            system_prompt = self.create_prompt()
            full_prompt = f"{system_prompt}\n\nUser request: {user_prompt}"
            
            response = self.model.generate_content(full_prompt)
            logger.info(f"Model response: {response.text}")
            
            # Parse the response
            func_name, params = self.parse_response(response.text)
            
            # Get and execute the function
            if func_name not in self.functions:
                raise ValueError(f"Function '{func_name}' not found")
            
            function = self.functions[func_name]
            result = function(**params)
            
            return result
            
        except Exception as e:
            logger.error(f"Error executing function: {e}")
            raise

# Example usage
def main():
    # Initialize model
    import os
    import google.generativeai as genai
    
    API_KEY = os.getenv("GOOGLE_API_KEY")
    if not API_KEY:
        raise ValueError("Missing GOOGLE_API_KEY in environment variables")
    
    genai.configure(api_key=API_KEY)
    model = genai.GenerativeModel("gemini-2.0-flash")
    
    # Create agent
    agent = SimpleFunctionAgent(model)
    
    # Register some functions
    def add_numbers(a: float, b: float) -> float:
        """Add two numbers together"""
        return a + b
    
    def multiply_numbers(a: float, b: float) -> float:
        """Multiply two numbers together"""
        return a * b
    
    def greet_person(name: str, language: str = 'English') -> str:
        """Greet a person in the specified language"""
        greetings = {
            'English': 'Hello',
            'Spanish': 'Hola',
            'French': 'Bonjour'
        }
        greeting = greetings.get(language, 'Hello')
        return f"{greeting}, {name}!"
    
    # Add functions to agent
    agent.add_function(add_numbers)
    agent.add_function(multiply_numbers)
    agent.add_function(greet_person)
    
    # Test the agent
    try:
        # Test math function
        result1 = agent.execute("add the numbers 10 and 20")
        print(f"Addition result: {result1}")
        
        # Test greeting function
        result2 = agent.execute("greet John in Spanish")
        print(f"Greeting result: {result2}")
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

2025-02-10 17:59:04,515 - INFO - Registered function: add_numbers
2025-02-10 17:59:04,516 - INFO - Registered function: multiply_numbers
2025-02-10 17:59:04,518 - INFO - Registered function: greet_person
2025-02-10 17:59:06,784 - INFO - Model response: FUNCTION: add_numbers
PARAMS: number1=10, number2=20

2025-02-10 17:59:06,785 - ERROR - Error executing function: main.<locals>.add_numbers() got an unexpected keyword argument 'number1'


Error: main.<locals>.add_numbers() got an unexpected keyword argument 'number1'
